## Preclass work for Session 3: Ensuring quality when training

Class notes of JWP on session 3
[https://jeongwoopark.notion.site/Ensuring-Quality-when-Training-0301cbfed4df491bb01f75f67d183fb6]

preclasswork descriptions and variable explanations in [https://github.com/minerva-university/cs156/tree/master/session03]

### Task 1: Regression with Facebook data

data retrieved from [https://docs.google.com/spreadsheets/d/1B2xkgo7clRcNRbIjBZT4MaleCxm6vjeZa5rhoAANo9E/edit#gid=1103908018]

In [24]:
#setting up the environment for task 1 (regression)
import numpy as np
import statsmodels.api as sm
import pandas as pd
import pystan
import scipy.stats as sts
from scipy import optimize
import math
import matplotlib.pyplot as plt
%matplotlib inline
import re
import seaborn as sns
from sklearn.metrics import r2_score

In [25]:
data = pd.read_csv(r'C:\Users\green\Downloads\facebook_train.csv')
print(data.head())

total_followers = data.total_followers
types = data.type
category = data.category
month = data.month
weekday = data.weekday
hour = data.hour
paid = data.paid
comment = data.comment
like = data.like
share = data.share

print(type(share[0]))
print(type(types[0]))


   total_followers   type  category  month  weekday  hour  paid  comment  \
0           124050  Photo         3      6        4    17     1        3   
1           132556  Photo         2      8        3    24     0       11   
2            92786  Photo         2      2        6    17     1        2   
3           135632  Photo         1      9        2     1     0        2   
4           139540   Link         1     12        3    17     1        7   

   like  share  
0    77     16  
1   313     50  
2   145     39  
3   299     49  
4   229     21  
<class 'numpy.int64'>
<class 'str'>


In [26]:
#Regression of comments
y_1 = comment
x_1 = [month,paid, total_followers, category, weekday, hour, like, share]

def reg(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    results = sm.OLS(y, X).fit()
    return results

print(reg(y_1, x_1).summary())

#regression of like
x_2 = [month,paid, total_followers, category, weekday, hour,comment, share]
y_2 = like
print(reg(y_2,x_2).summary())

#regression of shares
x_3 = [month,paid, total_followers, category, weekday, hour,comment, like]
y_3 = share

print(reg(y_3,x_3).summary())
#without categorical (discrete) variables with less than 3 values
#x_3 = [month, total_followers, weekday, hour,comment, like]
#y_3 = share

#print(reg(y_3,x_3).summary())
##suprisingly, this makes regression have less r^2

                            OLS Regression Results                            
Dep. Variable:                comment   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     179.5
Date:                Mon, 17 Jan 2022   Prob (F-statistic):          2.37e-125
Time:                        18:05:49   Log-Likelihood:                -1580.6
No. Observations:                 397   AIC:                             3179.
Df Residuals:                     388   BIC:                             3215.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.3052      0.031      9.992      0.0

So far, I've tried to make a regression and played around with it to continue and not continue some variables for the 4th regression. However, it seems not having discrete variable does not make r^2 go up even though those discrete variables have less than 3 values. 

This might mean (just a guess) that the categorical data with less than 3 values are highly related with the variables and thus giving a slight rise on r^2. 

In [27]:
data_test = pd.read_csv(r'C:\Users\green\Downloads\facebook_test.csv')



TypeError: 'RegressionResultsWrapper' object is not subscriptable

### Task 2: Casualty Classification

data retrieved from [https://docs.google.com/spreadsheets/d/1lOt3ktFmX0fiWa89dLUiKfv1zoYjyI23zo3Ntb6HzhQ/edit#gid=952175914]

In [4]:
task2 = pd.read_csv(r'C:\Users\green\Downloads\casualty_train.csv')
print(task2.head())

  casualty_class  gender age  severe pedestrian_location pedestrian_movement  \
0      passenger  female  33   False                 NaN                 NaN   
1      passenger  female  20   False                 NaN                 NaN   
2      passenger    male  52   False                 NaN                 NaN   
3      passenger  female  17   False                 NaN                 NaN   
4      passenger  female  20   False                 NaN                 NaN   

       travel  year  
0   motorbike  2007  
1         car  2005  
2         car  2006  
3  pedestrian  2012  
4   motorbike  2010  


Pedestrian_location and pedestrian movements have too much diversity to classify as well as having too much N/A. So, I am dropping it

Also, the year has to do nothing with the injuries. There can be a difference by change in law, however, that is more of previous/post legistration rather than specific year to do with the severeness of the injury. So, I am not using the variable for classification. 

casualty_class, age, gender, travel has a lot to do with the results. Because of difference in speed, force, protection, and body composition. 

In [11]:
casualty_class = task2['casualty_class']
gender = task2['gender']
age = task2['age']
travel = task2['travel']
severe = task2['severe']

print(len(travel))
print(gender.shape)
print(age.shape)
print(travel.shape)
print(severe.shape)

data2 = np.array([casualty_class, gender, age, travel])

232835
(232835,)
(232835,)
(232835,)
(232835,)


In [15]:
#setting up the environment
import pprint
from IPython.display import display, HTML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

In [19]:
#In the future, I gotta use kfold after the class to make the idea a bit more concrete to get higher accuracy
X = task2.drop(columns = ['year', 'pedestrian_movement', 'pedestrian_location'])
y = severe
trainData,testData,trainLabel,testLabel = train_test_split(X, y, test_size=0.2)

model = KNeighborsClassifier(n_neighbors = 482) #rule of thumb is data squared
from sklearn import metrics
scores = cross_val_score(model, X, y, cv=5, scoring = 'f1_macro')
model.fit(trainData,trainLabel)
predictions = model.predict(testData)
print(classification_report(testLabel, predictions))

#why is it not working???
newdata = pd.read_csv(r'C:\Users\green\Downloads\casualty_test.csv')
predictions = model.predict(newdata)
print(classification_report(testLabel, predictions))

C:\Users\green\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\green\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\green\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 198, in fit
    return self._fit(X, y)
  File "C:\Users\green\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 400, in _fit
    X, y = self._validate_data(X, y, accept_sparse="csr", multi_output=True)
  Fi

ValueError: could not convert string to float: 'Unknown'